<a href="https://colab.research.google.com/github/Sunnnyyy16/DHH_AI_Story/blob/main/%08Automated_StoryTelling_DHHKids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI-Based Automated Storybook Creation Tool for Deaf or Hard of Hearings Children

청각장애 아동 대상 AI 기반 동화제작 자동화 툴

# Part 1: ASL Translation to Standard English Sentence

# Part 2: Extract Image Depictions and Characters Appearance

In [ ]:
import transformers
import torch

MY_HF_TOKEN = "WRITE_HF_TOKEN"
# 모델 ID
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token = MY_HF_TOKEN
)

In [ ]:
# story 3파트로 나누기
story1 = """
There once was a boy who grew bored while watching over the village sheep.
He wanted to make things more exciting. So, he yelled out that he saw a wolf chasing the sheep.
All the villagers came running to drive the wolf away. However, they saw no wolf. The boy was amused, but the villagers were not.
They told him not to do it again.
"""

story2 = """
Shortly after, he repeated this antic. The villagers came running again, only to find that he was lying.
Later that day, the boy really sees a wolf sneaking amongst the flock. He jumped up and called out for help.
But no one came this time because they thought he was still joking around.
"""

story3 = """
At sunset, the villagers looked for the boy. He had not returned with their sheep.
They found him crying. He told them that there really was a wolf, and the entire flock was gone.
An old man came to comfort him and told him that nobody would believe a liar even when they are being honest.
"""

# 전체 스토리
full_story = """
There once was a boy who grew bored while watching over the village sheep.
He wanted to make things more exciting. So, he yelled out that he saw a wolf chasing the sheep.
All the villagers came running to drive the wolf away. However, they saw no wolf.
The boy was amused, but the villagers were not. They told him not to do it again. Shortly after, he repeated this antic.
The villagers came running again, only to find that he was lying. Later that day, the boy really sees a wolf sneaking amongst the flock.
He jumped up and called out for help. But no one came this time because they thought he was still joking around.
At sunset, the villagers looked for the boy. He had not returned with their sheep.
They found him crying. He told them that there really was a wolf, and the entire flock was gone.
An old man came to comfort him and told him that nobody would believe a liar even when they are being honest.
"""

In [ ]:
#### 외양묘사
# OSOS prompt 활용
messages = [
    {"role": "system", "content": "In the given story, assign main Characters Characteristics that seems the most normal and fitting to the story such as color and shape. If it is a human, descriptions should include hair color and clothes. For example: Boy:[Brown hair][Blue eyes][white T-shirt and jeans]"},
    {"role": "user", "content": f"Here is a story: {full_story}"}
]

# 결과 생성
outputs = pipeline(
    messages,
    max_new_tokens=256,
)

# 결과 출력
print(outputs[0]["generated_text"])

In [ ]:
### story 요약
# OSOS prompt 활용
messages = [
    {"role": "system", "content": "You are an AI assitant to describe image for storybook. Write one sentence that depicts an image of the story. The format should be [subject][doing action],[adjective],[background subject],[scenery]."},
    {"role": "user", "content": f"Here is a story: {story1}"}
]

# 결과 생성
outputs = pipeline(
    messages,
    max_new_tokens=256,
)

# 결과 출력
print(outputs[0]["generated_text"])

# Part 3: Image generation with fine-tuned diffusion model

In [ ]:
!pip install diffusers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes

In [ ]:
from diffusers import DiffusionPipeline

DF_HF_TOKEN = 'WRITE_HF_TOKEN'

pipe = DiffusionPipeline.from_pretrained(
    "dhlim55/kids-storybook_illustration",
    token = DF_HF_TOKEN)

pipe = pipe.to("cuda")

In [ ]:
def plot_images(images):
    from matplotlib import pyplot as plt
    plt.figure()
    f, axarr = plt.subplots(1, len(images), figsize=(20,10))
    for ax, img in zip(axarr.flatten(), images):
        ax.imshow(img)
        ax.axis('off')
    plt.show()

In [ ]:
# story1
prompt = """[Brown hair][Blue eyes][White T-shirt and jeans] A mischievous boy, with a sly grin, stands amidst a flock of sheep, with a distant village in the background, under a bright blue sky with fluffy white clouds."""

images = pipe(prompt, num_images_per_prompt=3).images

plot_images(images)

In [ ]:
# story2
prompt = """[Brown hair][Blue eyes][White T-shirt and jeans] A boy, laughing mischievously, lies on the ground amidst a flock of sheep, a sunny green meadow in the background."""

images = pipe(prompt, num_images_per_prompt=3).images

plot_images(images)

In [ ]:
# story3
prompt = """[Brown hair][Blue eyes][White T-shirt and jeans] A young boy, with tears streaming down his face, stands alone amidst the empty, grassy field, where a scattering of sheep's wool and a few stray sheep can be seen, with the setting sun casting a warm orange glow in the background."""

images = pipe(prompt, num_images_per_prompt=3).images

plot_images(images)